In [1]:
import math
import RegPattern
import re
import math
import codecs
from nltk.corpus import stopwords
from operator import itemgetter

In [2]:
def tokens(str1): 
	#stop_words = set(stopwords.words('english'))
	stop_words = []
	result = []
	for i in str1.strip().split(' '):
		# print( i )
		if i not in stop_words:
			result.append(i)
	if result == []:
		result = ['']
	return result

# to remove url
def del_url(line): #return re.sub(r'https?:\/\/.*', "", line).lower()
	result = ''
	# p_list = ["s'","'s"]
	for i in re.sub(r'https?:\/\/.*', "", line).lower().split(' '):
		if "s'" in i:
			result += i.replace("s'"," ")
		elif "'s" in i:
			result += i.replace("'s"," ")
		elif i.startswith("'"):
			result += i.replace("'"," ") + " "
		elif i.endswith("'"):
			result += i.replace("'"," ")
		else:
			result += i + " "
            
	return result

# print( (tokens(del_url(""))) )

def checktag(tweet):
	times = 0
	emp_result = ''
	result = ''
	for i in tweet.split(' '):
		if i.startswith('#'): #remove @ amd # (# result += '<HASHTAG>' + ' ' + i.replace('#',''))
			# result += '<HASHTAG>'
			result += '<HASHTAG>' + ' ' + i.replace('#','')
			result += ' '
			times += 1
		elif i.startswith('@'):
			result += '<user_name>'
			result += ' '
		elif 'pm' in i:
			result += i.replace('pm',' <PM>')#change the time format and add 'space' to be tokenized
			result += ' '
		else:
			result += i
			result += ' '
	if times >= 4:
		return emp_result
	return result

def removeRT(tweet): #if RT: then discard this tweet
	result = ''
	for i in tweet.split(' '):
		if i.startswith('rt') or i.startswith('RT'): #remove @ amd #
			break
		else:
			result += i
			result += ' '
	return result

reg = RegPattern.RegPattern()

def regTime(tweet):
	finalResult1 = ''
	result = (reg.get_pattern(tweet,'re_time'))
	# print( reg.re_time )
	# print( result )
	for i in result:
		finalResult1 += str(i) + ' '
	return finalResult1

def regPatterns(tweet):
	finalResult = ''
	result = (reg.get_pattern(tweet,'re_all'))
	# print( result )
	for i in result:
		finalResult += i + ' '
	return finalResult 

def sample_match(emoPath, meta):
	edges = {}
# 	print( "Processing "+emoPath )

	content_file = codecs.open(emoPath,"r",encoding='utf-8')
	out = codecs.open("./network/samples_"+meta,"w", "utf-8-sig")
	#emoji = load_list("./emo_list.txt","./XD_list.txt")#
	word_list = []
	for line in content_file:
		line = line.strip()
		for word in tokens(regPatterns(checktag(del_url(line)))):
			word_list.append(word)
			out.write(word)
			out.write("\n")
	if meta == "2":
		return word_list
	
	chbigram=list2bigram(word_list)
	bigramfreqdict=bigram2freqdict(chbigram)
	bigramfreqsorted=sorted(bigramfreqdict.items(), key=itemgetter(1), reverse=True)
	#print( bigramfreqsorted )

	maxi = 0.0
	foundMax = 0
	for (token,num) in bigramfreqsorted:
		if len(token[0]) != 0 and len(token[1]) != 0:
			w1 = token[0]
			w2 = token[1]
			if not foundMax:
				foundMax = 1
				maxi = float(num)
			#out.write("%s -> %s : %d - %f"%(token[0],token[1],num, float(num)/float(maxi)))
			#out.write("\n")
			edges["%s\t%s"%(w1,w2)]=float(num)/float(maxi)
	out.close()
	return edges;

def pattern_match(line):
	word_list = []
	line = line.strip()
	for word in tokens(regPatterns(checktag(del_url(line)))):
		word_list.append(word)
	return word_list


def list2freqdict(mylist):
	mydict=dict()
	for ch in mylist:
		mydict[ch]=mydict.get(ch,0)+1
	return mydict

def list2bigram(mylist):
	return [mylist[i:i+2] for i in range(0,len(mylist)-1)]

def bigram2freqdict(mybigram):
	mydict=dict()
	for (ch1,ch2) in mybigram:
		mydict[(ch1,ch2)]=mydict.get((ch1,ch2),0)+1
	return mydict

def replace_space(path):
	with codecs.open(path,"r", "utf-8") as content_file:
		content = content_file.read()
# 	print( content )
	content.replace(" ","_blank_")
	out = codecs.open("path","w", "utf-8-sig")
	out.write(content)
	out.close()

#sample_match("./network/test_data3/accurate.txt","test")